In [15]:
import pandas as pd
import os

# Paths
in_path = "in/tables/sales_order.csv"
out_path = "out/tables/sales_order.csv"
os.makedirs("out/tables", exist_ok=True)

# Read CSV
df = pd.read_csv(in_path)

# Data inspection
display(df.head())        # first rows - to see structure.
df.info()                 # to see column types and null values
print(df.isnull().sum())  # count missing

,pk_sales_order,created_at,currency,currency_rate,country_code,postal_code
0,1773b6bd4062d3805645553e1bff2849,2024-04-07 03:12:04.000,HUF,0.0026,HU,9437.0
1,4640a007be81c905de555addfc9067c9,2024-10-21 17:51:26.000,HUF,0.0025,HU,9422.0
2,48848489c69fa5d40e349243c0d1ddcc,2024-09-17 22:45:13.000,CZK,0.0397,CZ,19800.0
3,679cafb288112926c81f617a2f1e4f71,2024-09-17 14:21:53.000,HUF,0.0025,HU,9700.0
4,4a0c5231df766ac348a9aba2c681104a,2024-08-12 16:47:37.000,CZK,0.0396,CZ,51101.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pk_sales_order  30000 non-null  object 
 1   created_at      30000 non-null  object 
 2   currency        30000 non-null  object 
 3   currency_rate   30000 non-null  float64
 4   country_code    30000 non-null  object 
 5   postal_code     29994 non-null  float64
dtypes: float64(2), object(4)
memory usage: 1.4+ MB
pk_sales_order    0
created_at        0
currency          0
currency_rate     0
country_code      0
postal_code       6
dtype: int64


In [16]:
# Data cleaning

# Drop rows with missing postal_code
df = df.dropna(subset=["postal_code"])

# Check results
print(df.isnull().sum())

pk_sales_order    0
created_at        0
currency          0
currency_rate     0
country_code      0
postal_code       0
dtype: int64


In the dataset there are only 6 missing postal codes out of ~30,000 rows, which is less than 0.02% of all records.
From a statistical point of view, this fraction is negligible and does not affect the representativeness of the data.
Because of this, I decided to remove those rows: keeping them does not add value, while deleting them simplifies the dataset and ensures that every record has a valid postal code for later analysis.

In [17]:
# Save cleaned data
df.to_csv(out_path, index=False, encoding="utf-8")
print(f"Saved to: {out_path}")

Saved to: out/tables/sales_order.csv


In [18]:
# Paths
in_path = "in/tables/sales_items.csv"
out_path = "out/tables/sales_items.csv"
os.makedirs("out/tables", exist_ok=True)

# Read CSV
df = pd.read_csv(in_path)

# Data inspection
display(df.head())        # first rows - to see structure.
df.info()                 # to see column types and null values
print(df.isnull().sum())  # count missing

,pk_sales_order_item,fk_sales_order,fk_item,sold_qty,product_price_local_currency,product_cost_eur
0,11b1949dd15de1bd4c6cb05d7d38ab91,7f83e787d514b3b8570e71e62b7f6b8c,f1f7bad6de9395164b730ec1f5e6b9e7,1.0,0.00,0.2700
1,c9d37fbe347994949a3bb23d775f778a,416546d2de986e03c06022de8e3e275d,b5f2355fbeef87460a5c89305d133a87,1.0,0.00,0.2700
2,ac178513a8221603b136960e90cdf211,ab7a5eb44c0b230a136903f4d17a9489,051ef5187f9da3ccb3c451980b2ca0a3,1.0,0.00,0.2700
3,a353ac0739b0c07451d817bab8d63311,9c6c8360032d5b43cf0b20ffcc977359,aa36501e64569cf48680edccc853cb0d,1.0,1251.97,1.6500
4,97eea3a2d006657335300d9df7c8760b,edd0bc7e15177da9f6614a4fddaeedc0,05e518308f9af3519d315beb9ad69bdf,1.0,2.75,0.8975


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132564 entries, 0 to 132563
Data columns (total 6 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   pk_sales_order_item           132564 non-null  object 
 1   fk_sales_order                132564 non-null  object 
 2   fk_item                       132514 non-null  object 
 3   sold_qty                      132564 non-null  float64
 4   product_price_local_currency  132564 non-null  float64
 5   product_cost_eur              132564 non-null  float64
dtypes: float64(3), object(3)
memory usage: 6.1+ MB
pk_sales_order_item              0
fk_sales_order                   0
fk_item                         50
sold_qty                         0
product_price_local_currency     0
product_cost_eur                 0
dtype: int64


In [19]:
# replace missing fk_item with "Unknown"
df["fk_item"] = df["fk_item"].fillna("Unknown")

# check results
print("Missing values after replacement:")
print(df.isnull().sum())

Missing values after replacement:
pk_sales_order_item             0
fk_sales_order                  0
fk_item                         0
sold_qty                        0
product_price_local_currency    0
product_cost_eur                0
dtype: int64


The column **`fk_item`** links each order item to a product.  
About 50 rows (~0.17%) had missing values.  

Instead of dropping them, I replaced nulls with **`Unknown`**:  
- keeps all orders in the dataset,  
- makes missing cases explicit and easy to filter out in product-level analysis,  
- ensures BI tools show a clear "Unknown" category instead of silently losing rows.  

In [20]:
# Save cleaned data
df.to_csv(out_path, index=False, encoding="utf-8")
print(f"Saved to: {out_path}")

Saved to: out/tables/sales_items.csv
